In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
config = {'lr': 0.0012098123619624396,
 'layers': 2,
 'step_size': 21,
 'gamma': 0.5302067528042456,
 'bptt': 12,
 'dropout': 0.35583243487203325}

In [3]:
config['lr'] = config['lr'] * config['gamma'] **3
config['lr']

0.0001803241035696008

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/lga.csv')

In [6]:
dataset.shape

(8757, 1045)

In [7]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
1,2018-01-01,4,1,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2018-01-01,5,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

774

In [9]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 271)

In [10]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

257

In [11]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [12]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
#         train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq_x, train_seq_y ,train_label))
    return inout_seq

In [13]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=100, num_layers=1, dropout=0):
        super().__init__()
                
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.hidden_cell = (torch.zeros(num_layers,1,self.hidden_size).to(device),
                    torch.zeros(num_layers,1,self.hidden_size).to(device))
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)


    def forward(self, x):
        self.hidden_cell = (torch.zeros(self.num_layers,1,self.hidden_size).to(device),
                    torch.zeros(self.num_layers,1,self.hidden_size).to(device))
           
        lstm_out, self.hidden_cell = self.lstm(x.view(len(x) ,1, -1), self.hidden_cell)
        
        return lstm_out, self.hidden_cell

In [14]:
class GraphPrediction(nn.Module):
    def __init__(self, feat_size=1, hidden_layer_size=100, network_size=1, layers=1, communities=10, ensembles=1, dropout=0, at_mat=None):
        super().__init__()
        
        # aggregation
        if at_mat != None:
            self.attachment_matrix = torch.nn.Parameter(at_mat)
            self.attachment_matrix.requires_grad = False
        else:
            self.attachment_matrix = torch.nn.Parameter(torch.randn(network_size,communities))
            self.attachment_matrix.requires_grad = True
        
        lstm_input = communities + feat_size
        
        self.ensembles = ensembles
        self.lstms = nn.ModuleList()
        self.linears = nn.ModuleList()
        for i in range(ensembles):
             self.lstms.append(LSTM(input_size=lstm_input, hidden_size=hidden_layer_size, num_layers=layers))
             self.linears.append(nn.Linear(hidden_layer_size, network_size))
            

    def forward(self, input_seq, feat):
        
        w = F.softmax(self.attachment_matrix, dim=1)
        x = torch.matmul(input_seq, self.attachment_matrix)
        x = torch.cat((x,feat),axis=1)
        x = x.view(len(input_seq) ,1, -1)
        
        predictions = []
        for i in range(self.ensembles):
            if torch.randn(1) < 0.7 or i==0 or not self.training:
                lstm_out, self.hidden_cell = self.lstms[i](x)
                y = self.linears[i](lstm_out.view(len(input_seq), -1))
                predictions.append(y)
        
        predictions = torch.stack(predictions)
#         print(predictions.shape)
        return predictions

In [15]:
def evaluate(model):
    model.eval()
    prediction = []
    with torch.no_grad():
        for feat,seq, labels in test_inout_seq:
#             model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
#                             torch.zeros(layers, 1, model.hidden_layer_size))
            y = model(seq,feat).mean(dim=0)[-1]
    
            prediction.append(y)

    y_test_ = torch.stack([labels[-1] for feat,seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
    y_pred_ = torch.stack(prediction).detach().cpu().numpy()

    res = y_pred_ - y_test_
    r2 = r2_score(y_test_, y_pred_, multioutput='variance_weighted')
    rmse = mean_squared_error(y_test_, y_pred_)
    mae = mean_absolute_error(y_test_, y_pred_)
#     print("r2: ",r2)
    return (res, r2, rmse, mae)

In [16]:
def get_at_mat(targetColumns):
    comms = pd.read_csv('/home/urwa/Documents/side_projects/urban/UrbanTemporalNetworks/Data/ZonetoComm.csv')  
    communities = list(set(comms.start_community))

    mapping = dict(zip(comms.start_id, comms.start_community))
    comm_to_index = dict(zip(communities,range(len(communities))))
    col_to_index = dict(zip(targetColumns,range(len(targetColumns))))

    attach = torch.zeros(len(targetColumns), len(communities))

    for t_c in targetColumns:
        com = mapping[int(t_c)]
        x_i = col_to_index[t_c]
        y_i = comm_to_index[com]

        attach[x_i,y_i] = 1

    return attach

In [17]:
pretrained_weights = '/home/urwa/Documents/side_projects/urban/urban_traffic_prediciton/ensemble/bayesian_opt/lga.pt'

In [ ]:
bptt = config['bptt']

R2List = []
residual_list = []

for m in range(1,13):
    month_index  = pd.to_datetime(dataset.Date).dt.month == m
    
    print('-------------------------------------------------')
    print('-------------------------------------------------')
    print("Month: ", m)


    trainData = dataset[~month_index]
    testData = dataset[month_index]

    print("\n train test split")
    print(trainData.shape)
    print(testData.shape)


    print("\n ")
    X_train = trainData[features_cols].values
    X_train = torch.tensor(X_train).float().to(device)
    print(X_train.shape)

    y_train = trainData[targetColumns].values
    y_train = torch.tensor(y_train).float().to(device)
    print(y_train.shape)

    X_test = testData[features_cols].values
    X_test = torch.tensor(X_test).float().to(device)
    print(X_test.shape)

    y_test = testData[targetColumns].values
    y_test = torch.tensor(y_test).float().to(device)
    print(y_test.shape)
    
    
    train_inout_seq = create_inout_sequences(X_train,y_train, bptt)
    
    test_inout_seq = create_inout_sequences(X_test,y_test, bptt)
    print("\n sequences")
    print(train_inout_seq[0][0].shape,train_inout_seq[0][1].shape, train_inout_seq[0][2].shape)
    
    at_mat = get_at_mat(targetColumns)
    print("\nattachment matrix")
    print(at_mat.shape)
    
    
    layers = config['layers']
    communities = 24
    network_size = len(targetColumns)
    feat_size = len(features_cols)
    ensembles=10
    dropout = config['dropout']

    model = GraphPrediction(feat_size = feat_size, hidden_layer_size=100,
                 network_size=network_size, layers=layers,
                communities=communities, ensembles=ensembles, dropout=dropout, at_mat=at_mat).to(device)

    loss_function = nn.L1Loss()   
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=config['gamma'])
    print("\n model inititalized")
    
    model.load_state_dict(torch.load(pretrained_weights))
    print("\n model loaded")
    
    residual, r2, rmse, mae = evaluate(model)
    print("Pretraining R2: ",r2)
    
    
    best_r2 = 0
    best_residual = residual
#     torch.save(model.state_dict(), 'data/'+'lga_'+str(m)+'.pt')
#     np.save('data/'+'lga_'+str(m)+'.npy', best_residual)
    
    
    epochs = 15

    for i in range(epochs):
        model.train()
        for feat,seq, labels in train_inout_seq:
            optimizer.zero_grad()
    #         model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
    #                         torch.zeros(layers, 1, model.hidden_layer_size).to(device))

            y_pred = model(seq, feat)
            labels = labels.repeat(y_pred.shape[0],1,1)

            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()

        scheduler.step()
        residual, r2, rmse, mae = evaluate(model)
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')
        
        if r2 > best_r2:
            best_r2 = r2
            best_residual = residual
            torch.save(model.state_dict(), 'data/'+'lga_'+str(m)+'.pt')
            np.save('data/'+'lga_'+str(m)+'.npy', best_residual)


    print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')
    print("bet_r2: ", best_r2)

    R2List.append(best_r2)
    residual_list.append(best_residual)   

-------------------------------------------------
-------------------------------------------------
Month:  1

 train test split
(8016, 271)
(741, 271)

 
torch.Size([8016, 13])
torch.Size([8016, 257])
torch.Size([741, 13])
torch.Size([741, 257])

 sequences
torch.Size([12, 13]) torch.Size([12, 257]) torch.Size([12, 257])

attachment matrix
torch.Size([257, 24])

 model inititalized

 model loaded
Pretraining R2:  0.8431269396317222
epoch:   0 loss: 1.28851068 r2: 0.800 rmse: 3.004 mae: 0.759
epoch:   1 loss: 1.27635264 r2: 0.804 rmse: 2.932 mae: 0.754
epoch:   2 loss: 1.27958429 r2: 0.804 rmse: 2.942 mae: 0.756
epoch:   3 loss: 1.24049485 r2: 0.805 rmse: 2.918 mae: 0.755
epoch:   4 loss: 1.22784507 r2: 0.806 rmse: 2.909 mae: 0.755
epoch:   5 loss: 1.24204397 r2: 0.803 rmse: 2.948 mae: 0.758
epoch:   6 loss: 1.21571994 r2: 0.806 rmse: 2.909 mae: 0.756
epoch:   7 loss: 1.21204007 r2: 0.804 rmse: 2.933 mae: 0.757
epoch:   8 loss: 1.20673060 r2: 0.804 rmse: 2.932 mae: 0.758
epoch:   9 los

epoch:  12 loss: 1.17277563 r2: 0.832 rmse: 3.561 mae: 0.866
epoch:  13 loss: 1.17111087 r2: 0.832 rmse: 3.564 mae: 0.866
epoch:  14 loss: 1.14025342 r2: 0.831 rmse: 3.579 mae: 0.868
epoch:  14 loss: 1.1402534246
bet_r2:  0.8348994957079321
-------------------------------------------------
-------------------------------------------------
Month:  7

 train test split
(8013, 271)
(744, 271)

 
torch.Size([8013, 13])
torch.Size([8013, 257])
torch.Size([744, 13])
torch.Size([744, 257])

 sequences
torch.Size([12, 13]) torch.Size([12, 257]) torch.Size([12, 257])

attachment matrix
torch.Size([257, 24])

 model inititalized

 model loaded
Pretraining R2:  0.8382378031041606
epoch:   0 loss: 1.30665851 r2: 0.817 rmse: 2.911 mae: 0.821
epoch:   1 loss: 1.27128124 r2: 0.817 rmse: 2.906 mae: 0.822
epoch:   2 loss: 1.27771342 r2: 0.817 rmse: 2.916 mae: 0.823
epoch:   3 loss: 1.23989022 r2: 0.817 rmse: 2.913 mae: 0.824
epoch:   4 loss: 1.26124358 r2: 0.816 rmse: 2.925 mae: 0.824


In [ ]:
R2List

In [ ]:
len(set(attachment))

In [ ]:
# 2 linear
# 0.47

In [ ]:
# 1 linear
# 0.51

In [ ]:
# 1 linear + RELU
# 0.50

In [ ]:
# 1 linear bptt = 24
# 0.52

In [ ]:
#ensemble
#0.53228

In [ ]:
#  TODO
# other hubs
# ensemble model